# Applied Data Science Capstone Project
### Ambarish Ambuj

## Introduction

The severity code of the accident is typically set such that it represents the extent of damage caused by the accident. In an environment of limited resources, focusing more resources on preventing high severity accidents is one of the solutions to minimize the amount of damage with given resources. However, to do that, an understanding of the factors that affect the severity of the accident and the extent to which they affect the severity, is essential. Hence, with the given data about accident severity and some related parameters, this project tries to come up with a model to predict the impact of some key parameters such as accident location type, collission type, weather condition, road condition, lighting condition, number of persons involved in the collision etc. on the severity of the accident. The output of this model can provide policy inputs to the government to take specific actions to mitigate the causes that impact the accident severity the most.

## Data Description

The base data is taken from the example dataset provided in the course at the link https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
 We will first import the data as a dataframe to get a glimpse of the data.

Importing the necessary python libraries for the project

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

Importing the data from csv file

In [2]:
df = pd.read_csv('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv')

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Let us check the size of the file, the column names and some sample data to get basic understanding of the data.

In [3]:
df.shape

(194673, 38)

In [4]:
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 38 columns):
SEVERITYCODE      194673 non-null int64
X                 189339 non-null float64
Y                 189339 non-null float64
OBJECTID          194673 non-null int64
INCKEY            194673 non-null int64
COLDETKEY         194673 non-null int64
REPORTNO          194673 non-null object
STATUS            194673 non-null object
ADDRTYPE          192747 non-null object
INTKEY            65070 non-null float64
LOCATION          191996 non-null object
EXCEPTRSNCODE     84811 non-null object
EXCEPTRSNDESC     5638 non-null object
SEVERITYCODE.1    194673 non-null int64
SEVERITYDESC      194673 non-null object
COLLISIONTYPE     189769 non-null object
PERSONCOUNT       194673 non-null int64
PEDCOUNT          194673 non-null int64
PEDCYLCOUNT       194673 non-null int64
VEHCOUNT          194673 non-null int64
INCDATE           194673 non-null object
INCDTTM           194673 non-null obje

In [6]:
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


So, there are 194673 observations of incidents. There are 38 columns in the original dataset but as is evident from a preview of first 5 rows of the data, a column called 'SeverityCode' is repeated. So, there are 37 attributes for  194673 incidents. However, going back to our problem definition, not all 37 attributes are of our interest. We are only interested in exploring the impact of certain mitigable attributes on severity of the accident. So, based on the primary theoretical understanding, we select 'SEVERITYCODE' as the dependent variable and following variables as dependent variable:  
1. 'ADDRTYPE': A catagorical variable representing the type of location where incident took place. It may take the values of 'Intersection', 'Block' etc.  
2. 'COLLISIONTYPE': A categorical variable indicating the type of collision such as head-on, angle etc.
3. 'PERSONCOUNT': An integer representing number of persons involved in the collision.
4. 'PEDCOUNT': An integer representing number of pedestrians involved in the collision.
5. 'PEDCYLCOUNT': An integer representing the number of bicycles involved in the collision.
6. 'VEHCOUNT': An integer representing the number of vehicles involved in the collision.
7. 'WEATHER': A categorical variable describing whether the weather was cloudy or rainy etc. at the time of collision
8. 'ROADCOND': A categorical variable describing condition of the road i.e. dry or wet
9. 'LIGHTCOND': A categorical variable describing the lighting condition at the time of collision.

So, let's extract the one target and 9 predictor variables from the dataframe and store it in a new dataframe.

In [7]:
df1 = df[["SEVERITYCODE", "ADDRTYPE", "COLLISIONTYPE", "PERSONCOUNT", "PEDCOUNT", "PEDCYLCOUNT", "VEHCOUNT","WEATHER", "ROADCOND", "LIGHTCOND"]]
df1.head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,WEATHER,ROADCOND,LIGHTCOND
0,2,Intersection,Angles,2,0,0,2,Overcast,Wet,Daylight
1,1,Block,Sideswipe,2,0,0,2,Raining,Wet,Dark - Street Lights On
2,1,Block,Parked Car,4,0,0,3,Overcast,Dry,Daylight
3,1,Block,Other,3,0,0,3,Clear,Dry,Daylight
4,2,Intersection,Angles,2,0,0,2,Raining,Wet,Daylight


As we have a large number of data available with us, the best treatment of missing values is to drop them so that we don't have to guess the missing values and thereby affect the output. So, we will drop any rows with missing values.

In [13]:
df2 = df1.dropna(axis=0)
df2.shape

(187504, 10)

So, some of the rows were dropped and now we have 187504 observations with us to train, validate and test the model.  
Next steps would be to conduct the exploratory data analysis, convert the categorical variables to dummies, run a logistic regression and derive insights from the regression. These steps will be submitted in second part of the assignment.